Hey, I'm Jobert Gutierrez and hereafter you'll find the logic and code used to answer the first assignment in the program Machine Learning Operations Zoomcamp offered by Data Talks Club.

# Module 1 Homework: General Overview and foundations
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Question 1. Downloading the data
We'll use the same [NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

## Answer:

I downloaded the data using this code:

In [ ]:
# For January 2023:
python -m wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

# For February 2023:
python -m wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet


Then, I read the January file as:

In [34]:
# Calling the libraries used:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

In [19]:
df = pd.read_parquet(r'C:/Users/jober/OneDrive/Desktop/mlops-hw2025/Homework_week01/yellow_tripdata_2023-01.parquet')

The number of columns is identified using the function:

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Question 2. Computing duration
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

## Answer:

The duration of the trips are calculated:

In [21]:
# The amount of time for each ride is:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

# Expressed in mutes is:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

# The stard deviation of the ride's duration is: 
df.duration.std()

42.594351241920904

## Question 3. Dropping outliers
Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

## Answer:

The distribution of duration variables are:

In [22]:
# The number of registries before removing outliers is:
r_before = len(df)
print(f'the number of rows with outliers is: {r_before}')

# Removing outliers:
df = df[(df.duration >= 1) & (df.duration <= 60)]

# The number of rows without outliers is:
r_after = len(df)
print(f'the number of rows without outliers is: {r_after}')

the number of rows with outliers is: 3066766
the number of rows without outliers is: 3009173


In [26]:
fraction = r_after/r_before
print(f'Then, the fraction of the records left after removing the outliers is: {fraction}') 


Then, the fraction of the records left after removing the outliers is: 0.9812202822125979


## Question 4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

## Answer:

The dimensionality is calculated as follows:

In [37]:
# The categorical variables selected are:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
strings = ['VendorID', 'RatecodeID']

df[categorical] = df[categorical].astype(str)
df[strings] = df[strings].astype(str)

# Using the variables indicated, the train dictionary is:
train_dicts = df[categorical].to_dict(orient='records')

# The feature matrix is calculated as:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# The shape of this matrix is:
X_train.shape

(3009173, 515)

The number of columns of this matrix is __515 columns__.